This notebook contains examples of how to use the different functions of times2bright.

In [1]:
import brightway2 as bw
import pandas as pd
import seaborn as sns
import numpy as np

# Integrate 

In [2]:
from times2bright.integrate import *

this module is used to adapt activities of the LCA software brightway2 (https://brightwaylca.org/). To test it we need to select an activity for an existing project. As an example, we will modify activities of the LCA database Ecoinvent 3.5.

In [5]:
bw.projects.set_current('natem_ei35')

of the databases of this project, we use a generic version of the consequential version of ecoinvent 3.5

In [6]:
lcadb_ei35con=bw.Database('ei_35con')

find an activity to modify using list comprehension

In [7]:
car=[act for act in lcadb_ei35con 
 if 'transport' in act['name'] 
 and 'passenger' in act['name']
 and 'diesel' in act['name']
 and 'EURO 5' in act['name']
 and 'large size' in act['name']
 and act['location']=='RoW'
 and act['activity type']=='ordinary transforming activity'
]
car

['transport, passenger car, large size, diesel, EURO 5' (kilometer, RoW, None)]

validate that there's only one result and assign it to the variable _car_

In [8]:
car=check_query(car)

In [9]:
car

'transport, passenger car, large size, diesel, EURO 5' (kilometer, RoW, None)

`find_eflow` is used to identify biosphere exchanges related to fuel and biosphere emissions, it returns a list of exchanges with a name similar or equal to a given string. it has as inputs the bw activity and the name of the biosphere flow.

In [11]:
find_eflow(car,'Carbon dioxide')

1 flows with the name:
Carbon dioxide, fossil


[Exchange: 0.2186945944 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')) to 'transport, passenger car, large size, diesel, EURO 5' (kilometer, RoW, None)>]

`find_tflow` works very similar but for technosphere flows.

In [12]:
find_tflow(car,'diesel')

careful 2 flows with the name:
passenger car, diesel
diesel, low-sulfur


[Exchange: 0.0133333333333333 kilogram 'market for passenger car, diesel' (kilogram, GLO, None) to 'transport, passenger car, large size, diesel, EURO 5' (kilometer, RoW, None)>,
 Exchange: 0.06964796 kilogram 'market for diesel, low-sulfur' (kilogram, RoW, None) to 'transport, passenger car, large size, diesel, EURO 5' (kilometer, RoW, None)>]

Ups, by _diesel_ the function also picked up _market for passenger car, diesel_. we can use `literal==True` to specify that we want a exact name match 

In [13]:
find_tflow(car,'diesel, low-sulfur',literal=True)

[Exchange: 0.06964796 kilogram 'market for diesel, low-sulfur' (kilogram, RoW, None) to 'transport, passenger car, large size, diesel, EURO 5' (kilometer, RoW, None)>]

In [14]:
find_tflow(car,'diesel, low-sulfur',literal=True)[0]['name']

'diesel, low-sulfur'

**Determine the efficiency and its units** (the output divided by the input of fuel). This is done with the function `determine_eff`. Calculate efficiencies is useful to scale them after.

In [15]:
determine_eff(car,'diesel, low-sulfur')

fuel unit: kilogram
output unit: kilometer


14.357922328234741

**Determine the emission factor** (the amount of an emission divided by the amount of fuel consumed). This can be done with `determine_ef`

In [16]:
determine_ef(car,'diesel, low-sulfur','Carbon dioxide')

1 flows with the name:
Carbon dioxide, fossil
fuel unit: {'kilogram'}
output unit: {'kilogram'}


3.14

These functions are smart enough to calculate the efficiency correctly even if there are several entries of the same fuel (e.g. fuel diesel coming from global markets and some for reginal markets). This characteristic can be found on ecoinvent datasets, particularly in datasets representing the `rest of the world`

lets calculate the CO2eq emissions

In [17]:
ipcc2013=('IPCC 2013', 'climate change', 'GWP 100a')

In [18]:
lca_car=bw.LCA({car:1},ipcc2013)

In [19]:
lca_car.lci()
lca_car.lcia()
lca_car.score

0.37618541081170376

-----


### Activity modifications

now we are going to introduce some modifications to the activity, because I don't want to modify the database, I create a copy of the activity to be later erased.

In [20]:
car_modified=car.copy()

**Change fuel type**

lets change the fuel type, it could be diesel from a regional market. or even a totally different fuel.

lets try to put petrol on my diesel car

In [21]:
petrol_market=[act for act in lcadb_ei35con 
 if 'petrol, low-sulfur' in act['name'] 
 and act['location']=='RoW'
 and act['activity type']!='ordinary transforming activity'
]
petrol_market

['market for petrol, low-sulfur' (kilogram, RoW, None)]

In [22]:
petrol_market=check_query(petrol_market)
petrol_market

'market for petrol, low-sulfur' (kilogram, RoW, None)

In [23]:
car_modified=change_origin_tflow(car_modified, # activity
                                 'diesel, low-sulfur', # existing fuel
                                 petrol_market # new fuel
                                )

warning, substituting activity does not produce same product
diesel, low-sulfur
petrol, low-sulfur


ok, it lets me do it but it provides a warning, good to know!

In [24]:
lca_carmodified=bw.LCA({car_modified:1},ipcc2013)

In [25]:
lca_carmodified.lci()
lca_carmodified.lcia()
lca_carmodified.score

0.40412295913681456

We can see that the life cycle impact of the modified car is different now. It seems that per kg producing petrol has higher emissions. The software does not know that you will break the car if you put petrol on a diesel car :).

Now the activity has a new field _harmonisation_, a dict storing changes done the the activity.

In [26]:
car_modified['harmonisation']

{'tflow_origin': True}

**Change the amount of a technosphere flow (e.g. fuel use)**

In [27]:
old_eff=determine_eff(car_modified,'petrol')

fuel unit: kilogram
output unit: kilometer


In [28]:
old_eff

14.357922328234741

In [29]:
car_modified=scale_technosphere_flow(car_modified,
                                     'petrol, low-sulfur',
                                     0.5)

In [30]:
determine_eff(car_modified,'petrol')

fuel unit: kilogram
output unit: kilometer


28.715844656469482

now it runs the double of km with the same amount of fuel

In [31]:
determine_ef(car_modified,'petrol','Carbon dioxide')

1 flows with the name:
Carbon dioxide, fossil
fuel unit: {'kilogram'}
output unit: {'kilogram'}


6.28

The amount of CO2 emissions per unit of fuel is not double. It makes sense, we have halved the fuel consumption without touching the emissions.

now we would need to use `scale_biosphere_flow` to adjust the CO2 emissions (and all emissions related to combustion). 
this can be more simply done using `scale_fuel_cons`

In [32]:
car_modified.delete()

### Change fuel consumption (and associated emissions)

we go back to the diesel car.

In [33]:
car_modified=car.copy()

In [34]:
car_modified=scale_fuel_cons(car_modified,'diesel, low-sulfur',0.5)

In [35]:
lca_carmodified=bw.LCA({car_modified:1},ipcc2013)

In [36]:
lca_carmodified.lci()
lca_carmodified.lcia()
lca_carmodified.score

0.24017666452524555

In [37]:
f'Doubling efficiency reduces GHG by {lca_carmodified.score/lca_car.score:.2%}'

'Doubling efficiency reduces GHG by 63.85%'

it is normal that we reduce less than 50% the life-cycle impact, as other impacts such as the impact of car manufacturing are not affected.

In [38]:
determine_ef(car_modified,'diesel, low-sulfur','Carbon dioxide')

1 flows with the name:
Carbon dioxide, fossil
fuel unit: {'kilogram'}
output unit: {'kilogram'}


3.14

when the fuel consumption is adjusted the emission factor (emissions per unit of fuel) remain the same. This function scales all the emissions to air, assuming they are related to the fuel consumption.

In [39]:
determine_eff(car_modified,'diesel, low-sulfur')

fuel unit: kilogram
output unit: kilometer


28.715844656469482

In [40]:
car_modified.delete()

the efficiency is in effect double the value of the initial proxy.

# analyse 
in this module we have some functions to analyse LCA results.

In [42]:
from times2bright.analyse import *

going back to our original diesel car. Usually what we get is the score of the impact of total impact of the activity. 

In [43]:
lca_car.score

0.37618541081170376

Knowing a bit of the computational structure of LCA, we can calculate the impact of the emissions of a activity, excluding the impact of all the activities associated to it. In the case of a car, this would mean all the impacts of the emissions associated to the operation, but none related to other _technosphere flows_

In [44]:
direct_impact(lca_car)

0.2199322965450635

In [45]:
f'percentange of CO2 from operation: {direct_impact(lca_car)/lca_car.score:.2%}'

'percentange of CO2 from operation: 58.46%'

We can see that around 58% of the impact of the car is associated with the emissions of operating the car. One has to be careful with these calculations, and know how the datasets are structured. For example market datasets will have very little _direct_ impact, because it will have a lot of technosphere inputs, but very little or none emissions associated.

### find isic code

this is a very simple function, just because the isic code is not very easy to parse the way datasets are structured

In [46]:
car['classifications']

[('ISIC rev.4 ecoinvent', '4922:Other passenger land transport')]

In [47]:
find_isic(car)

('4922', 'Other passenger land transport')

separating the number and the description, we can later analyse the division associated with the isic (the first two digits) more easily.

## preprotimes

These functions are used to preprocess outputs from the TIMES model given by the shell `VEDA-BE`.